In [ ]:
!pip install pandas pyarrow
import pandas as pd
import numpy as np

*Checks the security of the API and the data extraction method.*


In [ ]:
def authentication():
    """
    Simulates an authentication check.

    Returns:
    - bool: True if authentication is successful, False otherwise.
    """
    return True

*Define a function to extract the data from the Parquet file and store in a dataframe*

In [ ]:
def api_call(file_path='/content/home_assignment_raw_data.parquet'):
    """
    Authenticates and reads data from a parquet file.

    Parameters:
    - file_path(str): Path to the parquet file.

    Returns:
    - pd.DataFrame: DataFrame containing the parquet file data if authentication is successful.

    Raises:
    - Exception: Raises an exception if authentication fails.
    """
    auth = authentication()
    if auth:
        df = pd.read_parquet(file_path)
        return df
    else:
        raise Exception("Authentication failed.")

**Convert data types of the columns**
- *converts the data types of the columns to appropriate data types.*

In [ ]:
def data_quality(df):
    """
    Ensures data quality by converting column types.

    Parameters:
    - df(pd.DataFrame): DataFrame to be processed.

    Returns:
    - pd.DataFrame: Processed DataFrame with corrected data types.
    """
    df['client_id'] = df['client_id'].astype(str)
    df['date'] = pd.to_datetime(df['date'], format='%m/%d/%Y')
    df['ext_dev_ref'] = df['ext_dev_ref'].astype(str)
    df['resolution'] = df['resolution'].astype(str)
    return df

**Split Resolution into Unit and Scale**
- Splits Resolution column into Scale and Units for better analytical and quering purpose.
- Flattening the "energy_consumption" list to separate columns to help to individually analyze time series data and helpful to quering when building customer insights.

In [ ]:
def feature_extraction(df):
    """
    Extracts features from the resolution and energy consumption columns.

    Parameters:
    - df (pd.DataFrame): DataFrame to be processed.

    Returns:
    - pd.DataFrame: DataFrame with extracted features.
    """
    df[['Scale', 'Unit']] = df['resolution'].str.extract(r'(\d+)(\D+)')
    df['Scale'] = df['Scale'].astype(int)
    new_df = pd.concat([df.drop(columns=['energy_consumption']),
                          pd.DataFrame(df['energy_consumption'].tolist()).add_prefix('Hour_')], axis=1)
    return new_df

In [ ]:
# Main execution
try:
    df = api_call()
    df = data_quality(df)
    df = feature_extraction(df)
    print(df.head())
except Exception as e:
    print(f"An error occurred: {e}")

  client_id       date   ext_dev_ref resolution  Scale Unit  Hour_0  Hour_1  \
0    111111 2022-01-01  ext_device_1      60min     60  min   100.0   100.0   
1    111111 2022-01-02  ext_device_1      60min     60  min   100.0   100.0   
2    111111 2022-01-03  ext_device_1      60min     60  min   100.0   100.0   
3    111111 2022-01-04  ext_device_1      60min     60  min   100.0   100.0   
4    111111 2022-01-05  ext_device_1      60min     60  min   100.0   100.0   

   Hour_2  Hour_3  ...  Hour_14  Hour_15  Hour_16  Hour_17  Hour_18  Hour_19  \
0   100.0   100.0  ...    100.0    100.0    100.0    100.0    100.0    100.0   
1   100.0   100.0  ...    100.0    100.0    100.0    100.0    100.0    100.0   
2   100.0   100.0  ...    100.0    100.0    100.0    100.0    100.0    100.0   
3   100.0   100.0  ...    100.0    100.0    100.0    100.0    100.0    100.0   
4   100.0   100.0  ...    100.0    100.0    100.0    100.0    100.0    100.0   

   Hour_20  Hour_21  Hour_22  Hour_23  
0   

In [ ]:
output_path = '/content/New_DF_data.csv'
df.to_csv(output_path, index=False)